In [254]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io
import os

In [255]:
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    # Warp source image to destination based on homography
    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
            
    # Prepare a mask representing region to copy from the warped image into the original frame.
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

In [256]:
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/' # เรียกค่าจาก การ calibration
#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')


In [257]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000) # กำหนดค่า ของ  ARUCO
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(1, 1, 0.182, 0.0075, AruCo_dict)

In [258]:
def write_text(img, pose,dy, text) : # สร้าง  write_text 
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (0,0,255), 2) # ฟอนต์ ขนาด สี ความหนา

In [259]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi') # เรียก Video
template_img = cv2.imread('./images/final_exam/Templates/Template-1.png') # เรียยก template_img
qr = cv2.imread('./images/QR-Code.jpg') # เรียก qr 
im_src_size = qr.shape[:2] # เก็บค่า im_src_size 
src_points = np.float32([[-0.182,-0.240], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ]) #ตำแหน่งของ qr
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY) # แปลง template_img ให้เป็น template_gray

In [260]:
while vid.isOpened() : # คำสั่งเปิด Video
    ret,  frame = vid.read()  # สร้าง frame 

    if ret :
        
        query_img = frame
        kernel = np.array([[0,0,0], [0, 1.5,0],[0,0,0]])  # สร้าง kernel เพื่อใช่ค่า ทำให้ภาพสว่าง 
        filter2d = cv2.filter2D(query_img, -1, kernel) # filter2D เก็บค่า query_img ใน

        # alpha = 1.23 #  กำหนดค่า  alpha  
        # beta = 75 # กำหนดค่า  beta
        # filter2d = cv2.convertScaleAbs (  query_img , alpha = alpha ,beta = beta) # ทำการ convertScaleAbs  ให้กับ query_img เก็บค่าใน filter2d
        query_gray = cv2.cvtColor(filter2d, cv2.COLOR_BGR2GRAY) # นำ filter2d แปลงเป็น ภาพ 
    
        sift = cv2.SIFT_create() #ใช้ function sift ค้นหาจุดที่คล้ายกันมากที่สุด 
        bf = cv2.BFMatcher() # การจับคู่ Brute-Force กับตัวบอก SIFT  

        template_kpts, template_desc = sift.detectAndCompute(template_img, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)

        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])
        

           ##### Ransac ##### 


        if len(good_matches) > 14 : # good matches = 14
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.2) # RANSAC 
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2) 
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines( frame, [np.int32(transformed_box)], True, (0,255,255), 2, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            augmented = augmented_image(detected_img, qr, src_points,transformed_box)

            img = augmented.copy() # สร้าง img เก็บค่า  augmented
            markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(augmented , AruCo_dict, parameters = AruCo_params)
        
            if len(markerCorners) > 0: # ส่วนของ ARUCO เพื่อสแกนหาตำแหน่ง
                img = cv2.aruco.drawDetectedMarkers(augmented , markerCorners)
                rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
                for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                    img = cv2.aruco.drawAxis(augmented , K, dist, rvec, tvec, 0.05)
                    x = tvec[0,0]
                    y = tvec[0,1]
                    z = tvec[0,2]
                    text = " X :{:.2f} \n Y :{:.2f} \n Z :{:.2f}".format( x, y, z) # ตำแหน่งทศนิยม 
                    cX = (corner[0,0][0] + corner[0,2][0])/1.8  # ตำแหน่งภาพที่จะจัดว่างตำแหน่ง
                    cY = (corner[0,0][1] + corner[0,2][1])/1.7  # ตำแหน่งภาพที่จะจัดว่างตำแหน่ง
                    write_text(detected_img, (cX, cY), 30, text) #เขียนใน detected_img
                ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 

            # cv2.imshow('Video frame',augmented)
            cv2.imshow('Video detected',detected_img)
           
        if cv2.waitKey(int(1000/20)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break

vid.release  ()
cv2.destroyAllWindows ()